In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import utils
from utils import ImagePair, ImageMatcher
import h5py
from scipy.spatial.transform import Rotation as Rot

In [3]:
DATA_DIR = '/Users/richard/Desktop/Dataset/'
frames = utils.load_video(DATA_DIR + 'f600.mp4', grayscale=True)[:100]
matcher = ImageMatcher(frames)

In [4]:
image_pairs = []

#interval = 15
#for i in range(0, len(frames) - interval, interval):
#    for j in range(i + interval, len(frames), interval):
#        p = matcher.match(i, j)
#        if p is None:
#            continue
#        image_pairs.append(p)

for i in range(0, len(frames) - 1):
    p = matcher.match(i, i+1)
    if p is None:
        print('No match found')
    image_pairs.append(p)


In [5]:
utils.export_image_pairs(DATA_DIR + f'cart10.h5', image_pairs)

In [13]:
cam_params_raw = utils.import_optimized_cam_params(DATA_DIR + 'optimized_cam_params.h5')

cam_params = {}
for i, p in cam_params_raw.items():
    cam_params[i] = {
        'R': Rot.from_rotvec(p[:3], degrees=False).as_matrix(),
        'hfov': 2 * np.degrees(np.arctan(320 / p[3])),
    }

In [14]:
trajs = np.load(DATA_DIR + 'f600.npy')
cam_params_gt = {}
for i in cam_params.keys():
    p = trajs[i]
    cam_params_gt[i] = {
        'R': Rot.from_euler('YXZ', p[:3], degrees=True).as_matrix(),
        'hfov': p[3],
    }

base_R = cam_params[0]['R'].T.copy()
for p in cam_params.values():
    p['R'] = base_R @ p['R']

base_R = cam_params_gt[0]['R'].T.copy()
for p in cam_params_gt.values():
    p['R'] = base_R @ p['R']

In [15]:
# Compute the angular pose error and hfov error between cam_params and cam_params_gt
angular_errors = []
hfov_errors = []
for i in cam_params.keys():
    R = cam_params[i]['R']
    R_gt = cam_params_gt[i]['R']
    angular_errors.append(np.degrees(Rot.from_matrix(R @ R_gt.T).magnitude()))
    hfov_errors.append(np.abs(cam_params[i]['hfov'] - cam_params_gt[i]['hfov']))

# pretty print the mean / max of angular and hfov errors to 5 sf
print(f'angular error mean: {np.mean(angular_errors):.5f}, max: {np.max(angular_errors):.5f}')
print(f'hfov error mean: {np.mean(hfov_errors):.5f}, max: {np.max(hfov_errors):.5f}')


angular error mean: 2.60801, max: 5.32236
hfov error mean: 0.48206, max: 0.92603


In [9]:
# print gt hfovs
print('gt hfovs:')
for i in cam_params_gt.keys():
    print(f'{i}: {cam_params_gt[i]["hfov"]:.5f}')

# print optimized hfovs
print('optimized hfovs:')
for i in cam_params.keys():
    print(f'{i}: {cam_params[i]["hfov"]:.5f}')


gt hfovs:
0: 75.00000
1: 75.00000
10: 75.00000
11: 75.00000
12: 75.00000
13: 75.00000
14: 75.00000
15: 75.00000
16: 75.00000
17: 75.00000
18: 75.00000
19: 75.00000
2: 75.00000
20: 75.00000
21: 75.00000
22: 75.00000
23: 75.00000
24: 75.00000
25: 75.00000
26: 75.00000
27: 75.00000
28: 75.00000
29: 75.00000
3: 75.00000
30: 75.00000
31: 75.00000
32: 75.00000
33: 75.00000
34: 75.00000
35: 75.00000
36: 75.00000
37: 75.00000
38: 75.00000
39: 75.00000
4: 75.00000
40: 75.00000
41: 75.00000
42: 75.00000
43: 75.00000
44: 75.00000
45: 75.00000
46: 75.00000
47: 75.00000
48: 75.00000
49: 75.00000
5: 75.00000
50: 75.00000
51: 75.00000
52: 75.00000
53: 75.00000
54: 75.00000
55: 75.00000
56: 75.00000
57: 75.00000
58: 75.00000
59: 74.00000
6: 75.00000
60: 73.00000
61: 72.00000
62: 71.00000
63: 70.00000
64: 69.00000
65: 68.00000
66: 67.00000
67: 66.00000
68: 65.00000
69: 64.00000
7: 75.00000
70: 63.00000
71: 62.00000
72: 61.00000
73: 60.00000
74: 59.00000
75: 58.00000
76: 57.00000
77: 56.00000
78: 55.000